In [16]:
import os
import keras
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [17]:
data = []
category = []

In [40]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/brain_tumor_dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
for filename in os.scandir("/content/drive/My Drive/brain_tumor_dataset/no"):
    if filename.is_file():
        #print(filename.path)
        img = Image.open(filename.path)
        img = img.resize(size=(32,32))
        img = img.convert('L')
        data.append(np.array(img).flatten())
        category.append(0)
        del img

In [78]:
for filename in os.scandir("/content/drive/My Drive/brain_tumor_dataset/yes"):
    if filename.is_file():
        #print(filename.path)
        img = Image.open(filename.path)
        img = img.resize(size=(32,32))
        img = img.convert('L')
        data.append(np.array(img).flatten())
        category.append(1)
        del img

In [79]:
x,y = np.array(data),category

In [44]:
x

array([[ 2,  2,  2, ...,  2,  2,  2],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 1,  1,  1, ...,  1,  1,  1],
       [29, 30, 30, ..., 27, 27, 38],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [80]:
x.shape

(759, 1024)

In [81]:
len(y)

759

In [82]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=20)

In [83]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()

In [84]:
lg.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [85]:
lg.score(x_train,y_train)

1.0

In [86]:
lg.score(x_test,y_test)

1.0

In [87]:
knn_model = KNeighborsClassifier(n_neighbors=7,weights="uniform",algorithm="auto")

In [88]:
knn_model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=7)

In [89]:
knn_model.score(x_train,y_train)

0.8747940691927513

In [90]:
knn_model.score(x_test,y_test)

0.8289473684210527

In [92]:
y_pred_train_knn = knn_model.predict(x_train)


In [94]:
train_accuracy = accuracy_score(y_train, y_pred_train_knn)
print(f'Train Accuracy: {train_accuracy}')

Train Accuracy: 0.8747940691927513


In [95]:
y_pred = knn_model.predict(x_test)


In [96]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8289473684210527
Confusion Matrix:
 [[48 12]
 [14 78]]
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.80      0.79        60
           1       0.87      0.85      0.86        92

    accuracy                           0.83       152
   macro avg       0.82      0.82      0.82       152
weighted avg       0.83      0.83      0.83       152



In [98]:
with open('brain_tumor1_SVM.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

In [99]:
def preprocess_image(file_path):
    image = cv2.imread(file_path)
    if image is None:
        raise ValueError(f"Could not read the image from path: {file_path}")
    image = cv2.resize(image, (32, 32))  # Resize to 128x128
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    return image.flatten()

In [106]:
uploaded_image_path = '/content/img1.jpg'
uploaded_image = preprocess_image(uploaded_image_path)

In [107]:
prediction = knn_model.predict(np.expand_dims(uploaded_image, axis=0))
print(prediction)
# Interpret the prediction
if prediction[0] == 0:
    print("Predicted: No Brain Tumor")
else:
    print("Predicted: Brain Tumor Detected")

[1]
Predicted: Brain Tumor Detected
